In [1]:
import logging

import probtorch
import torch

import combinators
import foldable
import hmm
import importance
import mcmc
import trace_tries
import utils

logging.basicConfig(format='%(asctime)s %(message)s', datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)

In [2]:
hmm_params = utils.vardict({
    'mu': {
        'loc': torch.arange(5, dtype=torch.float) * 2,
        'scale': torch.ones(5) * 0.25,
    },
    'sigma': {
        'loc': torch.ones(5),
        'scale': torch.ones(5) * 0.25,
    }
})
for k in range(6):
    hmm_params['Pi_%d' % k] = {'concentration': torch.ones(5)}

In [3]:
init_hmm = combinators.hyper_population(combinators.PrimitiveCall(hmm.init_hmm), (1,), hyper=hmm_params)
hmm_step = foldable.Foldable(combinators.PrimitiveCall(hmm.hmm_step), initializer=init_hmm)
hmm_run = foldable.Reduce(hmm_step, generator=lambda: range(50))

In [4]:
(z_last, mu, sigma, pi, pi0), generative = hmm_run()

In [5]:
data = {k.rpartition('/')[-1]: rv.value for (k, rv) in generative.filter(lambda k, rv: 'X_' in k)}

In [6]:
num_particles = 250

In [7]:
smc_hmm_params = utils.vardict({
    'mu': {
        'loc': torch.arange(5, dtype=torch.float) * 2,
        'scale': torch.ones(5),
    },
    'sigma': {
        'loc': torch.ones(5),
        'scale': torch.ones(5),
    }
})
for k in range(6):
    smc_hmm_params['Pi_%d' % k] = {'concentration': torch.ones(5)}

In [8]:
init_hmm = combinators.hyper_population(combinators.PrimitiveCall(hmm.init_hmm), (num_particles,), trainable=smc_hmm_params)

In [9]:
rmsmc_hmm = mcmc.reduce_resample_move_smc(combinators.PrimitiveCall(hmm.hmm_step), (num_particles,), lambda: range(50), initializer=init_hmm, moves=5)

In [10]:
samples, inference = rmsmc_hmm(data=data, trace=trace_tries.HierarchicalTrace())

In [11]:
for t in range(50):
    t += 1
    key = list(filter(lambda key: ('Z_%d' % t) in key, generative.keys()))[0]
    accuracy = (inference[key].value == generative[key].value).to(dtype=torch.float).mean()
    print('SMC accuracy at time %d: %f' % (t, accuracy))

SMC accuracy at time 1: 0.372000
SMC accuracy at time 2: 0.476000
SMC accuracy at time 3: 0.584000
SMC accuracy at time 4: 0.824000
SMC accuracy at time 5: 0.284000
SMC accuracy at time 6: 0.588000
SMC accuracy at time 7: 0.396000
SMC accuracy at time 8: 0.828000
SMC accuracy at time 9: 0.388000
SMC accuracy at time 10: 0.324000
SMC accuracy at time 11: 0.616000
SMC accuracy at time 12: 0.124000
SMC accuracy at time 13: 0.264000
SMC accuracy at time 14: 0.280000
SMC accuracy at time 15: 0.416000
SMC accuracy at time 16: 0.640000
SMC accuracy at time 17: 0.156000
SMC accuracy at time 18: 0.620000
SMC accuracy at time 19: 0.468000
SMC accuracy at time 20: 0.188000
SMC accuracy at time 21: 0.540000
SMC accuracy at time 22: 0.576000
SMC accuracy at time 23: 0.872000
SMC accuracy at time 24: 0.248000
SMC accuracy at time 25: 0.352000
SMC accuracy at time 26: 0.484000
SMC accuracy at time 27: 0.204000
SMC accuracy at time 28: 0.372000
SMC accuracy at time 29: 0.928000
SMC accuracy at time 30